In [ ]:
# Agent analitic plotlib5

#!pip install huggingface_hub
#!pip install langchain huggingface_hub transformers
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
#!pip install huggingface_hub[hf_xet]
!pip install hf_xet
#!pip install --upgrade transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer
!pip install torch
!pip install plotly


!pip install --upgrade transformers langchain huggingface_hub torch


#hf_pipeline = pipeline("text-generation", model="distilgpt2", max_new_tokens=100)

!pip install huggingface_hub[hf_xet]
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.tools import BaseTool
#from langchain.llms import HuggingFace
from transformers import pipeline
import pandas as pd
import glob
import os
from prophet import Prophet
import plotly.graph_objects as go
from typing import List, Tuple
import time
import logging
from dotenv import load_dotenv

from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.tools import BaseTool
from transformers import pipeline

# Schlüssel werden geladen von .env
load_dotenv(".env")

# Setting up the logger
logging.basicConfig(level=logging.INFO)

# class CSVForecastTool(BaseTool):CSV с Prophet
class CSVForecastTool(BaseTool):
    name: str = "CSVForecast"
    description: str = "Predicts time series from CSV files (columns Date и Close)"
    #Add an analysis tool CSV

    def _run(self, company_name: str) -> List[Tuple[str, go.Figure]]:
     #   company_folder = f"parsed/{company_name.lower()}/"
        company_folder = f"parsed/{company_name}/"

        if not os.path.exists(company_folder):
            return [(f"Error: Folder for company{company_name} not found.", None)]

        csv_files = glob.glob(os.path.join(company_folder, "*.csv"))
        if not csv_files:
            return [(f"Error: No CSV files for company{company_name}.", None)]

        results = []
        for file in csv_files:
            try:
                df = pd.read_csv(file)
                if "Date" not in df.columns or "Close" not in df.columns:
                    continue

                # Data transformation for Prophet
                df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
                df = df.dropna(subset=["Date", "Close"])
                df = df.rename(columns={"Date": "ds", "Close": "y"})

                # Forecasting with Prophett
                model = Prophet()
                model.fit(df)
                future = model.make_future_dataframe(periods=90)
                forecast = model.predict(future)

                # Zeichnen eines Diagramms
                fig = go.Figure()
                fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Forecast'))
                fig.add_trace(go.Scatter(x=df['ds'], y=df['y'], mode='markers', name='Historical data'))
                fig.update_layout(title=f"Forecast fur {company_name}", xaxis_title=Datum", yaxis_title="Closing price")

                # Text generation
                change = forecast['yhat'].iloc[-1] - forecast['yhat'].iloc[-91]
                change_pct = (change / forecast['yhat'].iloc[-91]) * 100
                trend = "Height" if change > 0 else "Fall"
                result_text = f"{company_name}: Прогноз на следующий квартал: {trend} ~{abs(change_pct):.2f}%"
                results.append((result_text, fig))

            except Exception as e:
                results.append((f"error while processing {file}: {str(e)}", None))

        return results

# Rechner-Tool
class CalculatorTool(BaseTool):
    name: str = "Calculator"
    description: str = "Führt mathematische Berechnungen durch"

    def _run(self, query: str) -> str:
        try:
            return str(eval(query))
        except Exception as e:
            return f"Rechenfehler: {str(e)}"

# Создание инструментов
csv_forecast_tool = CSVForecastTool()
calculator_tool = CalculatorTool()

tools = [
    Tool(name="CSVForecast", func=csv_forecast_tool._run, description="Прогноз по CSV для компании"),
    Tool(name="Calculator", func=calculator_tool._run, description="Математический калькулятор"),
   # Tool(name=csv_forecast_tool.name, func=csv_forecast_tool._run, description=csv_forecast_tool.description)
]

# Инициализация LLM с Hugging Face
# Загружаем модель и токенизатор
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

hf_pipeline = pipeline("text-generation", model="gpt2", max_new_tokens=100)

# Генерация текста
output = hf_pipeline("Hello, how are you?")
print(output[0]['generated_text'])

output = hf_pipeline("Hello, how are you?")
print(output)
#llm_hf = pipeline("text-generation", model="gpt2")                     #HuggingFace(pipeline=hf_pipeline)

# Обёртка в LangChain совместимый llm
llm = HuggingFacePipeline(pipeline=hf_pipeline)
# Инициализация агента
agent = initialize_agent(
    tools=tools,
    llm=llm,                            #llm_hf,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Пример вызова агента
query = "Проанализируй Nvidia и построй график"
response = agent.run(query)

query = "Проанализируй Nvidia за 6 месяцев, покажи график и спрогнозируй на следующий квартал."
response = agent.run(query)

# Проверка и отображение
if isinstance(response, list) and isinstance(response[0], tuple):
    text, fig = response[0]
    print(text)
    if fig:
        fig.show()
else:
    print("Ответ агента:", response)

#if isinstance(response, list):
#    for result in response:
 #       if isinstance(result, tuple):
  #          text, figure = result
  #          print(text)
   #         if figure:
    #            figure.show()  # Отображение графика
    #    else:
    #        print(result)  # Если это просто ошибка или информация
#else:
 #   print("Ответ от агента не является списком. Полученный ответ:", response)

# Пример запроса
query = "Сколько будет 2 + 2?"
response = agent.run(query)
print(response)

# Вывод результата
if isinstance(response, list):
    for result in response:
        if isinstance(result, tuple):
            text, figure = result
            print(text)
            if figure:
                figure.show()  # Отображение графика
        else:
            print(result)  # Если это просто ошибка или информация
else:
    print("Ответ от агента не является списком. Полученный ответ:", response)




from langgraph.prebuilt import create_react_agent

Data_Science_Agent = create_react_agent(llm, [retrieve])




